# Group 7 Lab 2

***
a) Suppose the input graph G = (V, E) is stored in an adjacency matrix and we 
use an array for the priority queue. Implement the Dijkstra’s algorithm using this 
setting and analyze its time complexity with respect to |V| and |E| both 
theoretically and empirically.

In [ ]:
# priority queue

class PriorityQueue(object):
    def __init__(self):
        self.queue = []
    
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
    
    # if empty return True
    def isEmpty(self):
        return len(self.queue) == 0
    
    # vertex is stored with distance
    # vertex 2 with distance 20: (2, 20)
    def insert(self, vertex):
        self.queue.append(vertex)
    
    def get_smallest(self):
        min_val = 1e7
        min_index = 0
        for i in range(len(self.queue)):
            if self.queue[i][1]<min_val:
                min_val = self.queue[i][1]
                min_index = i
        smallest = self.queue[min_index]
        del(self.queue[min_index])
        return smallest

    def remove_vertex(self, vertex):
        for i in range(len(self.queue)):
            if self.queue[i][0] == vertex:
                item = self.queue[i]
                del(self.queue[i])
                return item

In [ ]:
# graph for PQ

class PQGraph(object):
    def __init__(self, vertices):
        # self.V is the number of vertices
        # self.graph is the adjacency matrix
        self.V = vertices
        self.graph = [[0 for column in range(vertices+1)]
                      for row in range(vertices+1)]
    
    def dijkstra(self, src):
        # since source is the vertex num, we need to -1
        src = src - 1
        
        # set up distance = inf except source = 0
        dist = [1e7] * self.V
        dist[src] = 0
        
        # set up predecessor list
        pred = [-1] * self.V
        pred[src] = src + 1
        
        # set up sols
        sols = [False] * self.V
        sols[src] = True
        
        # set up priority queue
        Q = PriorityQueue()
        
        # flag for first run
        flag = True
        
        # for each adjacent vertex that is not 0, insert into queue
        for i in range(self.V):
            if self.graph[src][i] != 0:
                Q.insert((i+1, self.graph[0][i]))
            
        # while queue not empty:
        while Q.isEmpty() == False:
            # u is the next vertex with smallest distance
            if flag == True:
                u = src
                flag = False
            else:
                u = Q.get_smallest()[0]-1
            
            # putting u into solution set
            sols[u] = True
            
            # for each vertex adjacent to u (self.graph[] != 0)
            # that is not in solution set and (S[] == False)
            # if new distance is smaller than previous distance (d[v] < d[u] + self.graph[u][v])
            for v in range(self.V):
                if self.graph[u][v] != 0 and sols[v] == False and dist[v] > dist[u] + self.graph[u][v]:
                    Q.remove_vertex(v)
                    dist[v] = dist[u] + self.graph[u][v]
                    pred[v] = u + 1
                    Q.insert((v, dist[v]))
            # print(dist)
        print("Predecessors: {}".format(pred))
        print("Distance from source: {}".format(dist))


def graphtest():
    # g = Graph(4)
    # v1 = [0, 5, 1, 0] # vertex 1
    # v2 = [0, 0, 7, 2] # vertex 2
    # v3 = [0, 0, 0, 4] # vertex 3
    # v4 = [3, 1, 7, 0] # vertex 4
    # g.graph = [v1, v2, v3, v4]
    # g.dijkstra(4)
    
    g = PQGraph(5)
    v1 = [0, 2, 2, 1, 0]
    v2 = [1, 0, 0, 0, 4]
    v3 = [0, 2, 0, 3, 1]
    v4 = [0, 2, 0, 0, 1]
    v5 = [0, 3, 0, 0, 0]
    g.graph = [v1, v2, v3, v4, v5]
    g.dijkstra(5)

***
b) Suppose the input graph G = (V, E) is stored in an array of adjacency lists and 
we use a minimizing heap for the priority queue. Implement the Dijkstra’s 
algorithm using this setting and analyze its time complexity with respect to |V| 
and |E| both theoretically and empirically. 

In [28]:
# minimizing heap

from sys import maxsize

class heapnode(object):
    def __init__(self, vertex, weight):
        self.vertex = vertex
        self.weight = weight
    
    def getweight(self):
        return self.weight
    def setweight(self, weight):
        self.weight = weight
    def getvertex(self):
        return self.vertex
    def setvertex(self, vertex):
        self.vertex = vertex
    

class MinHeap:
  
    def __init__(self, maxsize):
        self.maxsize = maxsize
        self.size = 0
        self.Heap = [node(0,maxsize)]*(self.maxsize + 1)
        self.Heap[0] = node(0,-maxsize)
        self.FRONT = 1
  
    # Function to return the position of
    # parent for the node currently
    # at pos
    def parent(self, pos):
        return pos//2
  
    # Function to return the position of
    # the left child for the node currently
    # at pos
    def leftChild(self, pos):
        return 2 * pos
  
    # Function to return the position of
    # the right child for the node currently
    # at pos
    def rightChild(self, pos):
        return (2 * pos) + 1
  
    # Function that returns true if the passed
    # node is a leaf node
    def isLeaf(self, pos):
        return pos*2 > self.size
  
    # Function to swap two nodes of the heap
    def swap(self, fpos, spos):
        self.Heap[fpos], self.Heap[spos] = self.Heap[spos], self.Heap[fpos]
  
    # Function to heapify the node at pos
    def minHeapify(self, pos):
  
        # If the node is a non-leaf node and greater
        # than any of its child
        if not self.isLeaf(pos):
            if (self.Heap[pos].getweight() > self.Heap[self.leftChild(pos)].getweight() or 
               self.Heap[pos].getweight() > self.Heap[self.rightChild(pos)].getweight()):
  
                # Swap with the left child and heapify
                # the left child
                if self.Heap[self.leftChild(pos)].getweight() < self.Heap[self.rightChild(pos)].getweight():
                    self.swap(pos, self.leftChild(pos))
                    self.minHeapify(self.leftChild(pos))
  
                # Swap with the right child and heapify
                # the right child
                else:
                    self.swap(pos, self.rightChild(pos))
                    self.minHeapify(self.rightChild(pos))
  
    # Function to insert a node into the heap
    def insert(self, element):
        if self.size >= self.maxsize :
            return
        self.size+= 1
        self.Heap[self.size] = element
  
        current = self.size
  
        while self.Heap[current].getweight() < self.Heap[self.parent(current)].getweight():
            self.swap(current, self.parent(current))
            current = self.parent(current)
  
    # Function to print the contents of the heap
    def Print(self):
        for i in range(1, (self.size//2)+1):
            print(" PARENT : "+ str(self.Heap[i].getvertex())+" LEFT CHILD : "+ 
                                str(self.Heap[2 * i].getvertex())+" RIGHT CHILD : "+
                                str(self.Heap[2 * i + 1].getvertex()))
  
    # Function to build the min heap using
    # the minHeapify function
    def minHeap(self):
  
        for pos in range(self.size//2, 0, -1):
            self.minHeapify(pos)
  
    # Function to remove and return the minimum
    # element from the heap
    def remove(self):
  
        popped = self.Heap[self.FRONT]
        self.Heap[self.FRONT] = self.Heap[self.size]
        self.size-= 1
        self.minHeapify(self.FRONT)
        return popped


def heaptest():
    print('The minHeap is ')
    minHeap = MinHeap(10)
    minHeap.insert(heapnode(1,5))
    minHeap.insert(heapnode(2,2))
    minHeap.insert(heapnode(3,4))
    minHeap.insert(heapnode(4,1))
    minHeap.insert(heapnode(5,2))
    minHeap.minHeap()
    minHeap.Print()
    root = minHeap.remove()
    print("The Min weight is {}".format(root.getweight()))
    print("The Min vertex is {}".format(root.getvertex()))
    minHeap.Print()
    root = minHeap.remove()
    print("The Min weight is {}".format(root.getweight()))
    print("The Min vertex is {}".format(root.getvertex()))
    minHeap.Print()
    root = minHeap.remove()
    print("The Min weight is {}".format(root.getweight()))
    print("The Min vertex is {}".format(root.getvertex()))
    minHeap.Print()
    root = minHeap.remove()
    print("The Min weight is {}".format(root.getweight()))
    print("The Min vertex is {}".format(root.getvertex()))
    minHeap.Print()
    root = minHeap.remove()
    print("The Min weight is {}".format(root.getweight()))
    print("The Min vertex is {}".format(root.getvertex()))
    minHeap.Print()

# heaptest()

In [ ]:
# heap graph
# class listnode(object):
#     def __init__(self, vertex, next = None):
#         self.vertex = vertex
#         self.next = next
    
#     def getnext(self):
#         return self.next
#     def setnext(self, next):
#         self.next = next
#     def getvertex(self):
#         return self.vertex
#     def setvertex(self, vertex):
#         self.vertex = vertex

# 1 is pointing to 2 and 3 with weights of 3 and 4 respectively
# graph = {1:((2, 2), (3, 4))}

# class HGraph(object):
#     def __init__(self):
#         return

***
c) Compare the two implementations in (a) and (b). Discuss which implementation 
is better and in what circumstances. 